In [4]:
# === Fast Inference API (Optimized + Multi-pill):
#     YOLO → (조건부 전체이미지 / 소프트 크롭)
#     → ResNet(1324) 단일/다중 알약 분류 (dl_idx 계산식 기반) ===
# -*- coding: utf-8 -*-
import os
import json
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

from ultralytics import YOLO
from PIL import Image

# -------------------------------------------------
# 0) 경로 / 기본 설정
# -------------------------------------------------
DRIVE = "/content/drive/MyDrive"

# 테스트용 기본 이미지 경로 (필요하면 이 값만 바꿔서 사용)
IMG_PATH = "/content/drive/MyDrive/캡스톤_원천_데이터/TS_48_단일.zip/K-018357/K-018357_0_2_0_0_70_000_200.png"

BEST_YOLO      = os.path.join(DRIVE, "best.pt")
RESNET_1324_PT = os.path.join(DRIVE, "best_model_generalized.pth")
CLASS_JSON_1K  = os.path.join(DRIVE, "pill_label_path_sharp_score.json")
CLASS_JSON_324 = os.path.join(DRIVE, "class_mapping_from_cache_1324.json")

for p in [BEST_YOLO, RESNET_1324_PT]:
    assert os.path.exists(p), f"가중치 파일 없음: {p}"

YOLO_CONF  = 0.25
YOLO_IOU   = 0.45
YOLO_IMGSZ = 640

CROP_SIZE        = 224
MIN_BOX_SIDE_PX  = 40
FULL_IMAGE_AREA_RATIO_THRESHOLD = 0.65
SQUARE_SCALE     = 1.3

NUM_CLASSES  = 1324
LABEL_OFFSET = 1000

MAX_PILLS_MULTI = 4  # 다중 알약 모드에서 최대 알약 개수

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# -------------------------------------------------
# 1) 라벨 맵 로드 (class_idx → K-코드)
#    그리고 최종 dl_idx는 "K-xxxxxx"의 뒤 6자리 정수 - 1 로 계산
# -------------------------------------------------
def load_label_map_generic(json_path):
    """
    pill_label_path_sharp_score.json / class_mapping_from_cache_1324.json 을 읽어서
    class_idx → K코드(K-xxxxxx) 매핑을 만든다.
    값이 경로/파일명일 경우 파일명에서 K코드를 파싱해서 사용.
    """
    if not json_path or not os.path.exists(json_path):
        return {}
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if isinstance(data, dict) and "label_to_kcode" in data:
        data = data["label_to_kcode"]

    out = {}
    if isinstance(data, dict):
        for k, v in data.items():
            try:
                key_int = int(k)
            except:
                continue

            val_str = str(v)
            base = os.path.basename(val_str)   # 예: "K013101_0_2_0_1_90_020_200.png"
            first = base.split("_")[0]         # 예: "K013101" 또는 "K-009272"

            if first.startswith("K-") and len(first) == 8 and first[2:].isdigit():
                # 이미 "K-009272" 같은 형식
                kcode = first
            elif first.startswith("K") and len(first) == 7 and first[1:].isdigit():
                # "K013101" → "K-013101"
                kcode = "K-" + first[1:]
            else:
                # 그 외에는 그냥 값 그대로 사용 (fallback)
                kcode = first

            out[key_int] = kcode

    return out

LABEL_MAP_1K  = load_label_map_generic(CLASS_JSON_1K)   # 0..999 → K코드
LABEL_MAP_324 = load_label_map_generic(CLASS_JSON_324)  # 0..323 → K코드 (1000~1323용)

def class_idx_to_kcode(global_idx: int) -> str:
    """
    모델의 global class index → K코드(K-xxxxxx)로 변환
    """
    if global_idx < LABEL_OFFSET:
        return LABEL_MAP_1K.get(global_idx, f"imagenet_{global_idx}")
    local = global_idx - LABEL_OFFSET
    return LABEL_MAP_324.get(local, f"unknown_{local}")

def kcode_to_dl_idx(kcode: str) -> str:
    """
    K코드에서 dl_idx 계산:
      예) "K-009272" → 뒤 6자리 009272 → 9272 - 1 = 9271
    매핑 파일 없이 규칙으로만 계산.
    """
    if len(kcode) >= 7:
        tail = kcode[-6:]  # 끝 6자리
        if tail.isdigit():
            val = int(tail)
            dl_val = val - 1
            if dl_val >= 0:
                return str(dl_val)
    # 실패 시 그냥 원래 문자열 반환 (디버그용)
    return kcode

def idx_to_dl_idx(global_idx: int) -> str:
    """
    class index → K코드 → dl_idx 로 변환
    """
    kcode = class_idx_to_kcode(global_idx)
    return kcode_to_dl_idx(kcode)

# -------------------------------------------------
# 2) ResNet 1324 모델 + 전처리 (전역 1회 로드)
# -------------------------------------------------
def build_resnet_1324(num_classes=NUM_CLASSES, model_path=RESNET_1324_PT):
    model = models.resnet152(weights=None)
    in_f = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(in_f, num_classes)
    )

    state = torch.load(model_path, map_location="cpu")
    if isinstance(state, dict):
        if "model_state_dict" in state:
            state = state["model_state_dict"]
        elif "model" in state:
            state = state["model"]
    missing, unexpected = model.load_state_dict(state, strict=False)
    if missing or unexpected:
        print(f"ℹ️ state_dict load: missing={len(missing)}, unexpected={len(unexpected)}")

    model.to(DEVICE)
    model.eval()

    # GPU면 half precision으로 살짝 더 빠르게
    if DEVICE.type == "cuda":
        model.half()

    return model

# 🔥 전역에서 한 번만 생성해서 재사용
RESNET_MODEL = build_resnet_1324()

# 전역 transform (이미지 → 텐서)
base_transform = transforms.Compose([
    transforms.Resize((CROP_SIZE, CROP_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def preprocess_pil(pil_img):
    t = base_transform(pil_img)
    # 모델이 half면 입력도 half로 맞추기
    if DEVICE.type == "cuda":
        return t.half()
    return t

@torch.no_grad()
def predict_resnet_batch(pil_imgs, topk=5):
    """
    전역 RESNET_MODEL 사용.
    pil_imgs: [PIL.Image, ...]
    """
    if not pil_imgs:
        return []
    xs = [preprocess_pil(im) for im in pil_imgs]
    x = torch.stack(xs).to(DEVICE)

    if DEVICE.type == 'cuda':
        logits = RESNET_MODEL(x)
    else:
        with torch.amp.autocast(device_type='cpu'):
            logits = RESNET_MODEL(x)

    probs  = F.softmax(logits, dim=1)
    topk_prob, topk_idx = torch.topk(probs, topk, dim=1)

    all_results = []
    for i in range(probs.shape[0]):
        res_i = []
        for p, idx in zip(topk_prob[i].tolist(), topk_idx[i].tolist()):
            res_i.append({
                "idx": int(idx),
                "prob": float(p),
            })
        all_results.append(res_i)
    return all_results

# -------------------------------------------------
# 3) YOLO 감지 공통 로직 (전역 YOLO 1회 로드)
# -------------------------------------------------
YOLO_DEVICE = 0 if DEVICE.type == "cuda" else "cpu"
YOLO_MODEL = YOLO(BEST_YOLO)

def detect_yolo_boxes(img_path):
    """
    YOLO로 bbox + confidence 추출 공통 함수.
    return:
      img (PIL),
      boxes: [[x1,y1,x2,y2], ...],
      confs: [float, ...]
    """
    assert os.path.exists(img_path), f"이미지 없음: {img_path}"

    det = YOLO_MODEL(
        img_path,
        imgsz=YOLO_IMGSZ,
        conf=YOLO_CONF,
        iou=YOLO_IOU,
        device=YOLO_DEVICE,
        verbose=False
    )[0]

    img = Image.open(img_path).convert("RGB")
    W, H = img.size

    boxes = []
    confs = []

    if det.boxes is not None and len(det.boxes) > 0:
        xyxy = det.boxes.xyxy.cpu().numpy().tolist()
        conf = det.boxes.conf.cpu().numpy().tolist()
        for (b, c) in zip(xyxy, conf):
            x1, y1, x2, y2 = map(int, b)
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(W-1, x2)
            y2 = min(H-1, y2)
            if (x2-x1) >= MIN_BOX_SIDE_PX and (y2-y1) >= MIN_BOX_SIDE_PX:
                boxes.append([x1, y1, x2, y2])
                confs.append(float(c))

    return img, boxes, confs

def square_crop_from_bbox(pil_img, xyxy, scale=1.3):
    W, H = pil_img.size
    x1, y1, x2, y2 = xyxy
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    bw = x2 - x1
    bh = y2 - y1
    side = max(bw, bh) * scale

    # 최소 사이즈 보장
    side = max(side, MIN_BOX_SIDE_PX * 1.5)

    half = side / 2.0
    nx1 = int(round(cx - half))
    ny1 = int(round(cy - half))
    nx2 = int(round(cx + half))
    ny2 = int(round(cy + half))

    nx1 = max(0, nx1)
    ny1 = max(0, ny1)
    nx2 = min(W, nx2)
    ny2 = min(H, ny2)
    if nx2 <= nx1 or ny2 <= ny1:
        return None

    crop = pil_img.crop((nx1, ny1, nx2, ny2))
    return crop

# -------------------------------------------------
# 4-A) 단일 알약 모드: 이미지 전체 기준 Top-K (기존 infer_pill_image)
# -------------------------------------------------
def make_crops_for_single_mode(img_path):
    """
    단일 알약 또는 "실질적으로 하나만 있는" 이미지에 대해,
    - YOLO로 bbox 탐지
    - 단일 + 거의 전체 차지면 full 이미지 1장
    - 그 외엔 bbox마다 square crop
    """
    img, boxes, _ = detect_yolo_boxes(img_path)
    W, H = img.size
    img_area = W * H

    # 단일 큰 박스면 전체 이미지 사용
    use_full_image_only = False
    if len(boxes) == 1:
        x1, y1, x2, y2 = boxes[0]
        box_area = (x2-x1) * (y2-y1)
        area_ratio = box_area / float(img_area + 1e-9)
        if area_ratio >= FULL_IMAGE_AREA_RATIO_THRESHOLD:
            use_full_image_only = True

    crop_images = []
    if use_full_image_only:
        crop_images.append(img.copy())
    else:
        for bbox in boxes:
            sq = square_crop_from_bbox(img, bbox, scale=SQUARE_SCALE) if boxes else None
            if sq is None:
                continue
            crop_images.append(sq)

    if not crop_images:
        crop_images.append(img.copy())

    return crop_images

def infer_pill_image_single(img_path: str, topk_global: int = 5):
    """
    [기존 기능] 이미지 전체 기준으로 최종 Top-K만 뽑는 모드.
    여러 crop을 만든 뒤, class별 max-pooling으로 통합.
    """
    crops = make_crops_for_single_mode(img_path)
    batch_results = predict_resnet_batch(crops, topk=topk_global)

    # 클래스별 max-pooling
    agg_scores = {}  # class_idx → max_prob
    for crop_res in batch_results:
        for t in crop_res:
            idx = t["idx"]
            p   = t["prob"]
            if idx not in agg_scores or p > agg_scores[idx]:
                agg_scores[idx] = p

    if not agg_scores:
        return []

    sorted_pairs = sorted(agg_scores.items(), key=lambda x: x[1], reverse=True)[:topk_global]

    final_results = []
    for rank, (idx, prob) in enumerate(sorted_pairs, start=1):
        dl_idx = idx_to_dl_idx(idx)
        final_results.append({
            "rank":  rank,
            "idx":   idx,
            "dl_idx": dl_idx,
            "prob":  prob,
        })

    return final_results

# -------------------------------------------------
# 4-B) 다중 알약 모드: 최대 4개 알약 각각에 대한 Top-K
# -------------------------------------------------
def infer_pill_image_multi(img_path: str, max_pills: int = MAX_PILLS_MULTI, topk: int = 5):
    """
    이미지 안에 최대 max_pills 개의 알약이 있을 수 있는 경우:
      - YOLO로 bbox + conf 탐지
      - conf 기준으로 상위 max_pills 개 선택
      - 각 bbox마다 crop → ResNet Top-K
      - 알약별로 별도 결과 리턴

    return 형식:
      [
        {
          "pill_index": 0,
          "bbox": [x1, y1, x2, y2],
          "topk": [
            {"rank": 1, "idx": ..., "dl_idx": "...", "prob": ...},
            ...
          ]
        },
        ...
      ]
    """
    img, boxes, confs = detect_yolo_boxes(img_path)

    results = []

    # YOLO가 아무것도 못 잡았으면: 이미지 전체를 "pill_index 0"으로 간주
    if not boxes:
        crops = [img.copy()]
        batch_results = predict_resnet_batch(crops, topk=topk)
        if batch_results:
            pred = batch_results[0]
            topk_list = []
            for rank, t in enumerate(pred, start=1):
                dl_idx = idx_to_dl_idx(t["idx"])
                topk_list.append({
                    "rank":   rank,
                    "idx":    t["idx"],
                    "dl_idx": dl_idx,
                    "prob":   t["prob"],
                })
            results.append({
                "pill_index": 0,
                "bbox": None,
                "topk": topk_list,
            })
        return results

    # conf 기준으로 정렬 후 상위 max_pills 개 선택
    idx_sorted = sorted(range(len(boxes)), key=lambda i: confs[i], reverse=True)
    idx_keep   = idx_sorted[:max_pills]

    crops = []
    kept_boxes = []
    for i in idx_keep:
        bbox = boxes[i]
        sq = square_crop_from_bbox(img, bbox, scale=SQUARE_SCALE)
        if sq is None:
            continue
        crops.append(sq)
        kept_boxes.append(bbox)

    if not crops:
        # fallback: 전체 이미지 하나만
        crops = [img.copy()]
        kept_boxes = [None]

    batch_results = predict_resnet_batch(crops, topk=topk)

    for pill_idx, (bbox, pred) in enumerate(zip(kept_boxes, batch_results)):
        topk_list = []
        for rank, t in enumerate(pred, start=1):
            dl_idx = idx_to_dl_idx(t["idx"])
            topk_list.append({
                "rank":   rank,
                "idx":    t["idx"],
                "dl_idx": dl_idx,
                "prob":   t["prob"],
            })
        results.append({
            "pill_index": pill_idx,
            "bbox": bbox,
            "topk": topk_list,
        })

    return results

# -------------------------------------------------
# 5) 스크립트 직접 실행 시 동작 예시
# -------------------------------------------------
if __name__ == "__main__":
    assert os.path.exists(IMG_PATH), f"이미지 없음: {IMG_PATH}"

    print("\n=== [단일 모드] 이미지 전체 기준 최종 Top-5 ===")
    single_res = infer_pill_image_single(IMG_PATH, topk_global=5)
    for r in single_res:
        print(
            f"Rank {r['rank']}: "
            f"class_idx={r['idx']}, dl_idx={r['dl_idx']}, prob={r['prob']*100:.2f}%"
        )

    print("\n=== [다중 모드] 최대 4개 알약 각각에 대한 Top-5 ===")
    multi_res = infer_pill_image_multi(IMG_PATH, max_pills=4, topk=5)
    for pill in multi_res:
        print(f"\n[Pill #{pill['pill_index']}] bbox={pill['bbox']}")
        for t in pill["topk"]:
            print(
                f"  Rank {t['rank']}: "
                f"class_idx={t['idx']}, dl_idx={t['dl_idx']}, prob={t['prob']*100:.2f}%"
            )

    # 필요하면 JSON으로 직렬화해서 다른 API로 넘기면 됨:
    # print(json.dumps(multi_res, ensure_ascii=False, indent=2))


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Device: cpu

=== [단일 모드] 이미지 전체 기준 최종 Top-5 ===
Rank 1: class_idx=1054, dl_idx=18356, prob=99.61%
Rank 2: class_idx=1081, dl_idx=19231, prob=0.17%
Rank 3: class_idx=1201, dl_idx=13100, prob=0.02%
Rank 4: class_idx=1214, dl_idx=13390, prob=0.02%
Rank 5: class_idx=1287, dl_idx=10204, prob=0.01%

=== [다중 모드] 최대 4개 알약 각각에 대한 Top-5 ===

[Pill #0] bbox=None
  Rank 1: class_idx=1054, dl_idx=18356, prob=99.61%
  Rank 2: class_idx=1081, dl_idx=19231, prob=0.17%
  Rank 3: class_idx=1201, dl_idx=13100, prob=0.02%
  Rank 4: class_idx=1214, dl_idx=13390, prob=0.02%
  Rank 5: class_idx=1287, dl_idx=10204, prob=0.01%


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
